In [5]:
!pip install pandas numpy matplotlib

import pandas as pd
import numpy as np
from math import pi, sin, cos, tan, log

# Hardcoded Krishnan Kovil parameters (9.9252°N, 78.1511°E)
LOCATION = {
    'latitude': 9.9252,        # Degrees
    'altitude': 160,           # Meters
    'coastal': False           # Inland location
}

# Hardcoded climate parameters (March 2025 data for Madurai)
CLIMATE = {
    'tmin': 24.3,             # Minimum temperature (°C) [6]
    'tmax': 37.3,             # Maximum temperature (°C) [6]
    'dew_point': 21.5,        # Dew point temperature (°C) [4]
    'wind_speed': 2.9,        # 2m wind speed (m/s) [4]
    'sunshine_hours': 9,      # Daily sunshine hours [6]
    'actual_vp': 1.85,        # Actual vapour pressure (kPa) calculated from dew point
    'month': 3,               # March
    'day': 13                 # Current date
}

# Load synthetic datasets
crop_df = pd.read_csv('Synthetic_crop_data.csv')
soil_df = pd.read_csv('synthetic_soil_data.csv')

def calculate_solar_radiation(lat, day):
    """Calculate solar radiation using Madurai-specific parameters [3][5]"""
    G_sc = 0.0820  # Solar constant (MJ/m²/min)
    lat_rad = lat * (pi / 180)
    J = day

    # Solar declination [5]
    δ = 0.409 * sin((2 * pi * J / 365) - 1.39)

    # Sunset hour angle
    ω_s = np.arccos(-tan(lat_rad) * tan(δ))

    # Extraterrestrial radiation
    R_a = (24 * 60 / pi) * G_sc * (ω_s * sin(lat_rad) * sin(δ) + cos(lat_rad) * cos(δ) * sin(ω_s))

    # Actual solar radiation (using Angstrom formula) [3]
    n = CLIMATE['sunshine_hours']
    N = (24 / pi) * ω_s
    R_s = (0.25 + 0.5 * (n / N)) * R_a

    return R_s

def penman_monteith_eto():
    """FAO Penman-Monteith equation implementation for Krishnan Kovil [3][5]"""
    T = (CLIMATE['tmax'] + CLIMATE['tmin']) / 2
    u2 = CLIMATE['wind_speed']
    es = 0.6108 * np.exp((17.27 * T) / (T + 237.3))
    ea = CLIMATE['actual_vp']

    # Atmospheric pressure [5]
    P = 101.3 * ((293 - 0.0065 * LOCATION['altitude']) / 293) ** 5.26

    # Psychrometric constant [5]
    γ = 0.000665 * P

    # Slope of vapor pressure curve [5]
    Δ = (4098 * es) / (T + 237.3) ** 2

    # Net radiation calculation
    R_s = calculate_solar_radiation(LOCATION['latitude'], CLIMATE['day'])
    R_nl = 4.903e-9 * ((T + 273.16) ** 4) * (0.34 - 0.14 * np.sqrt(ea)) * (1.35 * (R_s / calculate_solar_radiation(LOCATION['latitude'], CLIMATE['day'])) - 0.35)
    R_n = R_s * 0.77 - R_nl

    # Soil heat flux (negligible for daily calculations) [3]
    G = 0

    # Final ETo calculation [5]
    eto_num = 0.408 * Δ * (R_n - G) + γ * (900 / (T + 273)) * u2 * (es - ea)
    eto_den = Δ + γ * (1 + 0.34 * u2)
    eto = eto_num / eto_den

    return eto

# Calculate daily ETo
daily_eto = penman_monteith_eto()
print(f"Daily Reference ET₀ for Krishnan Kovil: {daily_eto:.2f} mm/day")

# Integration with crop coefficients
def calculate_crop_water_requirement(crop_name, growth_stage):
    """Calculate crop-specific water requirement"""
    crop_data = crop_df[crop_df['Crop'] == crop_name].iloc[0]
    kc = crop_data[f'Kc_{growth_stage}']
    etc = daily_eto * kc
    return etc

# Example usage for Rice in initial stage
rice_water_req = calculate_crop_water_requirement('Rice', 'Initial')
print(f"Rice water requirement: {rice_water_req:.2f} mm/day")

# Soil integration
def available_water_capacity(soil_type):
    """Calculate available water capacity from soil properties [1][2]"""
    soil_data = soil_df[soil_df['Soil Type'] == soil_type].iloc[0]
    whc = soil_data['Water Holding Capacity (mm/m)']
    root_zone = crop_df[crop_df['Crop'] == 'Rice']['Root_Depth_Initial'].values[0] / 1000  # Convert mm to meters
    return whc * root_zone

# Irrigation recommendation system
def irrigation_recommendation(crop, soil, growth_stage):
    etc = calculate_crop_water_requirement(crop, growth_stage)
    awc = available_water_capacity(soil)

    # Simple irrigation logic (80% depletion threshold) [1]
    irrigation_needed = etc - (awc * 0.8)
    return max(0, irrigation_needed)

# Example recommendation
print("Irrigation Recommendation:")
print(f"{irrigation_recommendation('Rice', 'Red Soil', 'Initial'):.2f} mm required")


Daily Reference ET₀ for Krishnan Kovil: 6.24 mm/day


KeyError: 'Crop'

In [6]:
# Fixed column references based on your datasets
def calculate_crop_water_requirement(crop_name, growth_stage):
    """Calculate crop-specific water requirement with exact column names"""
    try:
        # Match column name exactly from your notebook
        crop_data = crop_df[crop_df['Crop'] == crop_name].iloc[0]

        # Use exact growth stage format from dataset
        kc_column = f'Kc_{growth_stage}'  # Matches 'Kc_Initial', 'Kc_Mid_Season' etc.
        return daily_eto * crop_data[kc_column]
    except KeyError as e:
        print(f"Missing column: {e}. Available columns: {crop_df.columns}")
        return None
    except IndexError:
        print(f"Crop '{crop_name}' not found. Available crops: {crop_df['Crop'].unique()}")
        return None

# Updated soil function with exact column names
def available_water_capacity(soil_type):
    """Calculate available water capacity with validation"""
    try:
        # Exact column name from soil dataset
        soil_data = soil_df[soil_df['Soil Type'] == soil_type].iloc[0]

        # Use exact column name from notebook
        whc = soil_data['Water Holding Capacity (mm/m)']

        # Get root depth from crop data (convert mm to meters)
        root_zone = crop_df[crop_df['Crop'] == 'Rice']['Root_Depth_Initial'].values[0] / 1000

        return whc * root_zone * 0.8  # 80% depletion
    except KeyError as e:
        print(f"Missing column: {e}. Available soil types: {soil_df['Soil Type'].unique()}")
        return None


In [7]:
# After loading CSVs, add:
print("Crop Data Sample:")
print(crop_df[['Crop', 'Kc_Initial', 'Root_Depth_Initial']].head())

print("\nSoil Data Sample:")
print(soil_df[['Soil Type', 'Water Holding Capacity (mm/m)']].head())


Crop Data Sample:


KeyError: "None of [Index(['Crop', 'Kc_Initial', 'Root_Depth_Initial'], dtype='object')] are in the [columns]"

In [10]:
import math
from datetime import datetime, timedelta

# ================== HARDCODED DATASETS ==================
soil_properties = {
    'Red Soil': {
        'water_holding_capacity': 250,
        'field_capacity': 22.5,
        'wilting_point': 12.5,
        'infiltration_rate': 2.0,
        'bulk_density': 1.3,
        'texture': {'sand': 70, 'silt': 10, 'clay': 20}
    },
    'Black Clayey Soil': {
        'water_holding_capacity': 400,
        'field_capacity': 35.0,
        'wilting_point': 17.5,
        'infiltration_rate': 0.75,
        'bulk_density': 1.4,
        'texture': {'sand': 40, 'silt': 20, 'clay': 40}
    },
    'Brown Soil': {
        'water_holding_capacity': 300,
        'field_capacity': 30.0,
        'wilting_point': 14.5,
        'infiltration_rate': 1.5,
        'bulk_density': 1.3,
        'texture': {'sand': 50, 'silt': 25, 'clay': 25}
    },
    'Alluvial Soil': {
        'water_holding_capacity': 350,
        'field_capacity': 32.0,
        'wilting_point': 15.0,
        'infiltration_rate': 2.5,
        'bulk_density': 1.25,
        'texture': {'sand': 45, 'silt': 35, 'clay': 20}
    }
}

crop_properties = {
    'Rice': {
        'kc_values': {'initial': 0.4, 'development': 0.7, 'mid_season': 1.15, 'late_season': 0.8},
        'growth_stages': {'initial': 30, 'development': 30, 'mid_season': 60, 'late_season': 30},
        'root_depth': {'initial': 0.2, 'development': 0.3, 'mid_season': 0.5, 'late_season': 0.9},
        'critical_depletion': {'initial': 0.3, 'development': 0.3, 'mid_season': 0.2, 'late_season': 0.2},
        'water_sensitivity': 0.9,
        'typical_yield': 5.5,
        'growing_seasons': ['June-Oct', 'Nov-March']
    },
    'Sugarcane': {
        'kc_values': {'initial': 0.4, 'development': 0.6, 'mid_season': 0.8, 'late_season': 0.9},
        'growth_stages': {'initial': 35, 'development': 60, 'mid_season': 180, 'late_season': 90},
        'root_depth': {'initial': 0.3, 'development': 0.5, 'mid_season': 0.8, 'late_season': 1.0},
        'critical_depletion': {'initial': 0.4, 'development': 0.35, 'mid_season': 0.3, 'late_season': 0.2},
        'water_sensitivity': 0.7,
        'typical_yield': 80.0,
        'growing_seasons': ['Dec-Jan']
    },
    'Groundnut': {
        'kc_values': {'initial': 0.4, 'development': 0.5, 'mid_season': 1.15, 'late_season': 0.6},
        'growth_stages': {'initial': 25, 'development': 35, 'mid_season': 45, 'late_season': 25},
        'root_depth': {'initial': 0.3, 'development': 0.4, 'mid_season': 0.5, 'late_season': 0.7},
        'critical_depletion': {'initial': 0.45, 'development': 0.35, 'mid_season': 0.25, 'late_season': 0.2},
        'water_sensitivity': 0.8,
        'typical_yield': 2.5,
        'growing_seasons': ['June-Sep', 'Jan-April']
    },
    'Cotton': {
        'kc_values': {'initial': 0.45, 'development': 0.55, 'mid_season': 0.85, 'late_season': 0.7},
        'growth_stages': {'initial': 30, 'development': 50, 'mid_season': 60, 'late_season': 55},
        'root_depth': {'initial': 0.3, 'development': 0.5, 'mid_season': 0.9, 'late_season': 1.0},
        'critical_depletion': {'initial': 0.5, 'development': 0.4, 'mid_season': 0.3, 'late_season': 0.3},
        'water_sensitivity': 0.6,
        'typical_yield': 2.0,
        'growing_seasons': ['June-Oct']
    },
    'Banana': {
        'kc_values': {'initial': 0.5, 'development': 0.7, 'mid_season': 1.1, 'late_season': 0.8},
        'growth_stages': {'initial': 40, 'development': 80, 'mid_season': 120, 'late_season': 60},
        'root_depth': {'initial': 0.2, 'development': 0.4, 'mid_season': 0.6, 'late_season': 0.7},
        'critical_depletion': {'initial': 0.4, 'development': 0.35, 'mid_season': 0.3, 'late_season': 0.3},
        'water_sensitivity': 0.85,
        'typical_yield': 40.0,
        'growing_seasons': ['Year-round']
    }
}

krishnan_kovil_constants = {
    'latitude': 9.2088,
    'longitude': 77.2561,
    'elevation': 150,
    'reference_et': 5.2,
    'rainfall_pattern': {
        'annual': 850,
        'monsoon_months': [9, 10, 11, 12],
        'dry_months': [1, 2, 3, 4, 5]
    },
    'temperature': {
        'annual_min': 22,
        'annual_max': 36,
        'hottest_months': [4, 5],
        'coolest_months': [11, 12]
    }
}

# ================== CALCULATION FUNCTIONS ==================
def calculate_eto(tmin, tmax, elevation, lat, doy, wind_speed=2.0, rh_min=45, rh_max=75):
    """FAO Penman-Monteith equation implementation"""
    lat_rad = math.radians(lat)
    tmean = (tmax + tmin) / 2

    # Saturation vapor pressure
    es_tmin = 0.6108 * math.exp(17.27 * tmin / (tmin + 237.3))
    es_tmax = 0.6108 * math.exp(17.27 * tmax / (tmax + 237.3))
    es = (es_tmin + es_tmax) / 2

    # Actual vapor pressure
    ea = (es_tmin * (rh_max/100) + es_tmax * (rh_min/100)) / 2

    # Atmospheric parameters
    delta = 4098 * es / ((tmean + 237.3)**2)
    P = 101.3 * ((293 - 0.0065 * elevation)/293)**5.26
    gamma = 0.665e-3 * P

    # Solar calculations
    dr = 1 + 0.033 * math.cos(2 * math.pi * doy / 365)
    delta_rad = 0.409 * math.sin(2 * math.pi * doy / 365 - 1.39)
    ws = math.acos(-math.tan(lat_rad) * math.tan(delta_rad))

    # Radiation components
    Ra = (24*60/math.pi) * 0.0820 * dr * (
        ws * math.sin(lat_rad) * math.sin(delta_rad) +
        math.cos(lat_rad) * math.cos(delta_rad) * math.sin(ws)
    )
    Rso = (0.75 + 2e-5 * elevation) * Ra
    Rs = Ra * 0.5  # Estimated solar radiation

    # Net radiation
    Rns = (1 - 0.23) * Rs
    Rnl = 4.903e-9 * ((tmax + 273.16)**4 + (tmin + 273.16)**4)/2 * (0.34 - 0.14 * math.sqrt(ea)) * (1.35 * Rs/Rso - 0.35)
    Rn = Rns - Rnl

    # Final ETo calculation
    numerator = 0.408 * delta * Rn + gamma * (900/(tmean + 273)) * wind_speed * (es - ea)
    denominator = delta + gamma * (1 + 0.34 * wind_speed)
    eto = numerator / denominator

    return max(eto, 0)

def get_growth_stage(crop, day):
    """Determine current growth stage for given day"""
    stages = list(crop['growth_stages'].items())
    cumulative = 0
    for stage, duration in stages:
        cumulative += duration
        if day < cumulative:
            return stage
    return stages[-1][0]

def calculate_crop_water(crop_name, soil_type, planting_date, area_ha=1):
    """Main calculation function for crop water requirements"""
    crop = crop_properties[crop_name]
    soil = soil_properties[soil_type]

    current_date = datetime.strptime(planting_date, "%Y-%m-%d")
    total_days = sum(crop['growth_stages'].values())

    # Initialize soil moisture storage (mm)
    root_depth = crop['root_depth']['initial']
    available_water_max = (soil['field_capacity'] - soil['wilting_point'])/100 * root_depth*1000
    current_storage = available_water_max  # Start at field capacity

    irrigation_schedule = []
    total_water = 0

    for day in range(total_days):
        # Update growth parameters
        stage = get_growth_stage(crop, day)
        kc = crop['kc_values'][stage]
        root_depth = crop['root_depth'][stage]
        critical_depletion = crop['critical_depletion'][stage]

        # Recalculate max available water for current root depth
        available_water_max = (soil['field_capacity'] - soil['wilting_point'])/100 * root_depth*1000

        # Calculate ET components
        doy = current_date.timetuple().tm_yday
        eto = calculate_eto(
            tmin=krishnan_kovil_constants['temperature']['annual_min'],
            tmax=krishnan_kovil_constants['temperature']['annual_max'],
            elevation=krishnan_kovil_constants['elevation'],
            lat=krishnan_kovil_constants['latitude'],
            doy=doy
        )
        etc = eto * kc

        # Update soil moisture
        current_storage -= etc

        # Calculate depletion percentage
        depletion = 1 - (current_storage / available_water_max)

        # Check irrigation need
        if depletion > critical_depletion:
            irrigation_needed = available_water_max - current_storage
            irrigation_schedule.append({
                'day_num': day+1,
                'date': current_date.strftime("%Y-%m-%d"),
                'amount_mm': round(irrigation_needed, 1),
                'stage': stage
            })
            # Reset soil moisture after irrigation
            current_storage = available_water_max

        total_water += etc
        current_date += timedelta(days=1)

    return {
        'total_water_mm': round(total_water, 1),
        'total_water_kl': round(total_water * area_ha * 10, 1),
        'irrigation_count': len(irrigation_schedule),
        'daily_avg': round(total_water / total_days, 1),
        'schedule': irrigation_schedule
    }

# ================== EXAMPLE USAGE ==================
if __name__ == "__main__":
    # Test case 1: Rice in Alluvial Soil
    rice_result = calculate_crop_water(
        crop_name="Rice",
        soil_type="Alluvial Soil",
        planting_date="2024-06-01"
    )

    print("\n=== RICE CULTIVATION ===")
    print(f"Total Water Requirement: {rice_result['total_water_mm']} mm")
    print(f"Irrigation Events: {rice_result['irrigation_count']}")
    if rice_result['irrigation_count'] > 0:
        print(f"First Irrigation: {rice_result['schedule'][0]['date']} ({rice_result['schedule'][0]['amount_mm']} mm)")
    else:
        print("No irrigation needed - natural soil moisture sufficient")

    # Test case 2: Banana in Black Clayey Soil
    banana_result = calculate_crop_water(
        crop_name="Banana",
        soil_type="Black Clayey Soil",
        planting_date="2024-01-01"
    )

    print("\n=== BANANA PLANTATION ===")
    print(f"Total Water Requirement: {banana_result['total_water_mm']} mm")
    print(f"Irrigation Events: {banana_result['irrigation_count']}")
    if banana_result['irrigation_count'] > 0:
        print(f"First Irrigation: {banana_result['schedule'][0]['date']} ({banana_result['schedule'][0]['amount_mm']} mm)")

    print("\n=== SC PLANTATION ===")
    sc_result = calculate_crop_water(
        crop_name="Sugarcane",
        soil_type="Black Clayey Soil",
        planting_date="2024-12-01"
    )
    print(f"Total Water: {sc_result['total_water_mm']} mm")
    print(f"Irrigation Events: {sc_result['irrigation_count']}")
    if sc_result['irrigation_count'] > 0:
        print(f"First Irrigation: {sc_result['schedule'][0]['date']} ({sc_result['schedule'][0]['amount_mm']} mm)")

    print("\n=== GN PLANTATION ===")
    gn_result = calculate_crop_water(
        crop_name="Groundnut",
        soil_type="Red Soil",
        planting_date="2024-06-15"
    )
    print(f"Total Water: {gn_result['total_water_mm']} mm")
    print(f"Irrigation Events: {gn_result['irrigation_count']}")
    if gn_result['irrigation_count'] > 0:
        print(f"First Irrigation: {gn_result['schedule'][0]['date']} ({gn_result['schedule'][0]['amount_mm']} mm)")

    print("\n=== Cotton PLANTATION ===")
    cotton_result = calculate_crop_water(
        crop_name="Cotton",
        soil_type="Alluvial Soil",
        planting_date="2024-03-01"
    )
    print(f"Total Water: {cotton_result['total_water_mm']} mm")
    print(f"Irrigation Events: {cotton_result['irrigation_count']}")
    if gn_result['irrigation_count'] > 0:
        print(f"First Irrigation: {cotton_result['schedule'][0]['date']} ({cotton_result['schedule'][0]['amount_mm']} mm)")





=== RICE CULTIVATION ===
Total Water Requirement: 673.4 mm
Irrigation Events: 36
First Irrigation: 2024-06-05 (10.7 mm)

=== BANANA PLANTATION ===
Total Water Requirement: 1362.3 mm
Irrigation Events: 45
First Irrigation: 2024-01-06 (14.4 mm)

=== SC PLANTATION ===
Total Water: 1441.9 mm
Irrigation Events: 39
First Irrigation: 2024-12-11 (21.0 mm)

=== GN PLANTATION ===
Total Water: 504.9 mm
Irrigation Events: 29
First Irrigation: 2024-06-21 (14.9 mm)

=== Cotton PLANTATION ===
Total Water: 703.2 mm
Irrigation Events: 17
First Irrigation: 2024-03-11 (26.3 mm)
